In [115]:
"""
SISTEMA DE CLUSTERING PARA RECOMENDACIÓN DE MEDICAMENTOS POR CUM
================================================================

FLUJO USUARIO FINAL:
- Input: CUM (ej: "20015204-5") 
- Output: Lista de CUMs válidos similares

MANDATO: El sistema DEBE funcionar con CUMs reales para usuarios reales
"""

from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_samples
)
from sklearn.cluster import (
    KMeans,
    DBSCAN,
    MiniBatchKMeans
)
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import time
import logging
from datetime import datetime

warnings.filterwarnings('ignore')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('clustering_medicamentos_cum.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("🔬 SISTEMA DE RECOMENDACIÓN DE MEDICAMENTOS POR CUM")
print("="*60)
print("🎯 Input: CUM → Output: CUMs válidos similares")
print("="*60)

🔬 SISTEMA DE RECOMENDACIÓN DE MEDICAMENTOS POR CUM
🎯 Input: CUM → Output: CUMs válidos similares


In [116]:
# ============================================================================
# 1. CARGA DEL DATASET CON CUM PRESERVADO
# ============================================================================
logger.info("Cargando dataset con CUM...")

path_encoded = Path("./data/medicamentos_train_preprocesados.parquet")

if not path_encoded.exists():
   raise FileNotFoundError(f"No se encontró: {path_encoded}")

start_time = time.time()
df_encoded = pd.read_parquet(path_encoded)
load_time = time.time() - start_time

print(f"📁 Dataset cargado en {load_time:.2f}s")
print(f"📊 Shape: {df_encoded.shape}")

# VERIFICAR CUM
if 'CUM' not in df_encoded.columns:
   raise ValueError("❌ CUM NO ENCONTRADO - PROCESO DETENIDO")

print(f"✅ CUM encontrado: {df_encoded['CUM'].nunique():,} únicos")

2025-06-19 01:40:39,309 - INFO - Cargando dataset con CUM...


📁 Dataset cargado en 0.12s
📊 Shape: (248635, 13)
✅ CUM encontrado: 155,341 únicos


In [117]:
# 2. PREPARACIÓN DE DATOS PARA CLUSTERING
# ================|============================================================

print("\n🚀 PREPARANDO DATOS PARA CLUSTERING...")

# Separar CUM para el sistema de recomendación
cum_series = df_encoded['CUM'].copy()

# Features para clustering (todo excepto CUM)
feature_cols = [col for col in df_encoded.columns if col not in ['CUM', 'ESTADO REGISTRO', 'ESTADO CUM', 'MUESTRA MÉDICA']]
X = df_encoded[feature_cols].values

print(f"📊 Matriz features: {X.shape}")
print(f"📊 CUMs preservados: {len(cum_series):,}")
print(f"📊 Columnas para clustering: {feature_cols}")

# Limpiar datos problemáticos
n_nans = np.isnan(X).sum()
n_infs = np.isinf(X).sum()

if n_nans > 0 or n_infs > 0:
   print(f"⚠️ Limpiando {n_nans} NaN y {n_infs} infinitos...")
   X = np.nan_to_num(X, nan=0.0, posinf=1.0, neginf=-1.0)


🚀 PREPARANDO DATOS PARA CLUSTERING...
📊 Matriz features: (248635, 9)
📊 CUMs preservados: 248,635
📊 Columnas para clustering: ['ATC', 'VÍA ADMINISTRACIÓN', 'FORMA FARMACÉUTICA', 'PRINCIPIO ACTIVO', 'CANTIDAD CUM', 'UNIDAD', 'CANTIDAD', 'UNIDAD MEDIDA', 'EXPEDIENTE CUM']


In [139]:
# ANTES del loop de entrenamiento, añade esto:

print("\n🎯 APLICANDO PESOS JERÁRQUICOS A CARACTERÍSTICAS...")

# Crear matriz con pesos jerárquicos
X_weighted = df_encoded[feature_cols].copy()

# NIVEL 1 - OBLIGATORIOS (peso 10)
print("📊 Nivel 1 (Obligatorios) - peso 10x:")
if 'ATC' in X_weighted.columns:
   X_weighted['ATC'] *= 10
   print("   - ATC: 10x")
if 'VÍA ADMINISTRACIÓN' in X_weighted.columns:
   X_weighted['VÍA ADMINISTRACIÓN'] *= 10
   print("   - VÍA ADMINISTRACIÓN: 10x")

# NIVEL 2 - IMPORTANTES (peso 5)
print("📊 Nivel 2 (Importantes) - peso 5x:")
if 'PRINCIPIO ACTIVO' in X_weighted.columns:
   X_weighted['PRINCIPIO ACTIVO'] *= 5
   print("   - PRINCIPIO ACTIVO: 5x")
if 'FORMA FARMACÉUTICA' in X_weighted.columns:
   X_weighted['FORMA FARMACÉUTICA'] *= 5
   print("   - FORMA FARMACÉUTICA: 5x")

# NIVEL 3 - PERMISIVOS (peso 1 - sin cambio)
print("📊 Nivel 3 (Permisivos) - peso 1x:")
nivel3_cols = ['CANTIDAD CUM', 'UNIDAD', 'CANTIDAD', 'UNIDAD MEDIDA', 'EXPEDIENTE CUM', 'ESTADO REGISTRO', 'ESTADO CUM', 'MUESTRA MÉDICA']
for col in nivel3_cols:
   if col in X_weighted.columns:
       print(f"   - {col}: 1x")

# Convertir a numpy array
X = X_weighted.values

print(f"✅ Matriz con pesos aplicados: {X.shape}")
print(f"✅ Rango de valores ponderados: [{X.min():.3f}, {X.max():.3f}]")

# TU algoritmos_config queda igual - no cambies nada


🎯 APLICANDO PESOS JERÁRQUICOS A CARACTERÍSTICAS...
📊 Nivel 1 (Obligatorios) - peso 10x:
   - ATC: 10x
   - VÍA ADMINISTRACIÓN: 10x
📊 Nivel 2 (Importantes) - peso 5x:
   - PRINCIPIO ACTIVO: 5x
   - FORMA FARMACÉUTICA: 5x
📊 Nivel 3 (Permisivos) - peso 1x:
   - CANTIDAD CUM: 1x
   - UNIDAD: 1x
   - CANTIDAD: 1x
   - UNIDAD MEDIDA: 1x
   - EXPEDIENTE CUM: 1x
✅ Matriz con pesos aplicados: (248635, 9)
✅ Rango de valores ponderados: [-0.092, 101605.001]


In [140]:
# 3. CONFIGURACIÓN DE ALGORITMOS
# ============================================================================

print("\n🎯 CONFIGURANDO ALGORITMOS...")

# algoritmos_config = {
#     'kmeans': {
#         'estimator': KMeans,
#         'params': {
#             'n_clusters': 100,
#             'random_state': 123,
#             'n_init': 10,
#             'max_iter': 300
#         }
#     },
#     'mini_kmeans': {
#         'estimator': MiniBatchKMeans,
#         'params': {
#             'n_clusters': 100,
#             'random_state': 123,
#             'batch_size': 1000,
#             'max_iter': 300
#         }
#     },
#     'dbscan': {
#         'estimator': DBSCAN,
#         'params': {
#             'eps': 2.0,
#             'min_samples': 50,
#             'n_jobs': -1
#         }
#     }
# }

algoritmos_config = {
#    'kmeans_15': {
#        'estimator': KMeans,
#        'params': {
#            'n_clusters': 500,
#            'random_state': 123,
#            'n_init': 10,
#            'max_iter': 300
#        }
#    },
#    'kmeans_20': {
#        'estimator': KMeans,
#        'params': {
#            'n_clusters': 750,
#            'random_state': 123,
#            'n_init': 10,
#            'max_iter': 300
#        }
#    },
#    'kmeans_25': {
#        'estimator': KMeans,
#        'params': {
#            'n_clusters': 1000,
#            'random_state': 123,
#            'n_init': 10,
#            'max_iter': 300
#        }
#    },
   
    'dbscan': {
        'estimator': DBSCAN,
        'params': {
            'eps': 3.0,
            'min_samples': 100,
            'n_jobs': -1
        }
    },
    'dbscan_estricto': {
        'estimator': DBSCAN,
        'params': {
            'eps': 1.0,        # MUY ESTRICTO = patrones muy similares
            'min_samples': 100, # DENSO = grupos farmacológicamente coherentes
            'n_jobs': -1
    }
}
}

print(f"🔬 Algoritmos: {list(algoritmos_config.keys())}")


🎯 CONFIGURANDO ALGORITMOS...
🔬 Algoritmos: ['dbscan', 'dbscan_estricto']


In [141]:
# 4. ENTRENAMIENTO Y COMPARACIÓN
# ============================================================================

print("\n🧪 ENTRENANDO ALGORITMOS...")

resultados = {}
comparacion = []

for nombre, config in algoritmos_config.items():
   print(f"\n🔬 Entrenando {nombre.upper()}...")
   
   try:
       start = time.time()
       modelo = config['estimator'](**config['params'])
       labels = modelo.fit_predict(X)
       tiempo = time.time() - start
       
       n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
       n_noise = list(labels).count(-1) if -1 in labels else 0
       
       print(f"✅ Completado en {tiempo:.2f}s")
       print(f"📊 Clusters: {n_clusters}, Ruido: {n_noise}")
       
       if n_clusters > 1:
           # Calcular métricas
           if n_noise > 0:
               mask_clean = labels != -1
               X_clean = X[mask_clean]
               labels_clean = labels[mask_clean]
           else:
               X_clean = X
               labels_clean = labels
           
           calinski = calinski_harabasz_score(X_clean, labels_clean)
           davies = davies_bouldin_score(X_clean, labels_clean)
           
           print(f"📈 Calinski-H: {calinski:.2f}")
           print(f"📈 Davies-B: {davies:.4f}")
           
           resultados[nombre] = {
               'modelo': modelo,
               'labels': labels,
               'tiempo': tiempo,
               'n_clusters': n_clusters,
               'n_noise': n_noise,
               'calinski': calinski,
               'davies': davies
           }
           
           comparacion.append({
               'Algoritmo': nombre.upper(),
               'Tiempo_s': round(tiempo, 2),
               'Clusters': n_clusters,
               'Ruido_%': round((n_noise/len(labels)*100), 1),
               'Calinski': round(calinski, 2),
               'Davies': round(davies, 4)
           })
           
   except Exception as e:
       print(f"❌ Error: {str(e)}")
       logger.error(f"Error en {nombre}: {str(e)}")



🧪 ENTRENANDO ALGORITMOS...

🔬 Entrenando DBSCAN...
✅ Completado en 1.43s
📊 Clusters: 29, Ruido: 243437
📈 Calinski-H: 460877514.93
📈 Davies-B: 0.0051

🔬 Entrenando DBSCAN_ESTRICTO...
✅ Completado en 1.45s
📊 Clusters: 26, Ruido: 243953
📈 Calinski-H: 4423786492.57
📈 Davies-B: 0.0024


In [142]:
# 5. SELECCIÓN DEL MEJOR MODELO
# ============================================================================

print("\n📊 SELECCIONANDO MEJOR MODELO...")

if comparacion:
   df_comp = pd.DataFrame(comparacion)
   print("\n📋 COMPARACIÓN:")
   print(df_comp.to_string(index=False))
   
   # Ordenar por Calinski-Harabasz (mayor es mejor)
   df_comp = df_comp.sort_values('Calinski', ascending=False)
   mejor_algo = df_comp.iloc[0]['Algoritmo'].lower()
   
   print(f"\n🥇 MEJOR: {mejor_algo.upper()}")
   print(f"📈 Calinski: {df_comp.iloc[0]['Calinski']}")
   
   # Obtener mejor modelo y labels
   mejor_modelo = resultados[mejor_algo]['modelo']
   mejores_labels = resultados[mejor_algo]['labels']
   
   # Crear DataFrame final con CUM + características + cluster
   df_final = df_encoded.copy()
   df_final['Cluster'] = mejores_labels
   
   print(f"✅ DataFrame final creado: {df_final.shape}")
   
else:
   raise ValueError("❌ Ningún algoritmo funcionó")



📊 SELECCIONANDO MEJOR MODELO...

📋 COMPARACIÓN:
      Algoritmo  Tiempo_s  Clusters  Ruido_%     Calinski  Davies
         DBSCAN      1.43        29     97.9 4.608775e+08  0.0051
DBSCAN_ESTRICTO      1.45        26     98.1 4.423786e+09  0.0024

🥇 MEJOR: DBSCAN_ESTRICTO
📈 Calinski: 4423786492.57
✅ DataFrame final creado: (248635, 14)


In [143]:
# 6. FUNCIONES PARA SISTEMA CUM → CUMs
# ============================================================================

def buscar_cum_en_dataset(cum_input, df_final):
   """Busca un CUM en el dataset y retorna su índice."""
   matches = df_final[df_final['CUM'] == cum_input]
   if len(matches) > 0:
       return matches.index[0]
   return None

def recomendar_cums_similares(cum_input, df_final, n_recomendaciones=5):
   """
   FUNCIÓN PRINCIPAL: CUM → Lista de CUMs similares
   """
   try:
       # Buscar CUM en dataset
       indice = buscar_cum_en_dataset(cum_input, df_final)
       
       if indice is None:
           return {
               "error": f"CUM {cum_input} no encontrado en dataset",
               "cum_consultado": cum_input,
               "recomendaciones": []
           }
       
       # Obtener información del medicamento
       medicamento = df_final.iloc[indice]
       cluster_id = medicamento['Cluster']
       
       # Si está en ruido, no hay recomendaciones
       if cluster_id == -1:
           return {
               "cum_consultado": cum_input,
               "cluster": -1,
               "mensaje": "CUM en cluster de ruido - sin similares",
               "recomendaciones": []
           }
       
       # Buscar CUMs en el mismo cluster
       medicamentos_cluster = df_final[
           (df_final['Cluster'] == cluster_id) &
           (df_final.index != indice)  # Excluir el mismo medicamento
       ]
       
       if len(medicamentos_cluster) == 0:
           return {
               "cum_consultado": cum_input,
               "cluster": int(cluster_id),
               "mensaje": "No hay otros CUMs en el mismo cluster",
               "recomendaciones": []
           }
       
       # Obtener top N recomendaciones
       top_recomendaciones = medicamentos_cluster.head(n_recomendaciones)
       cums_recomendados = top_recomendaciones['CUM'].tolist()
       
       return {
           "cum_consultado": cum_input,
           "cluster": int(cluster_id),
           "total_cluster": len(medicamentos_cluster),
           "recomendaciones": cums_recomendados,
           "mensaje": f"Se encontraron {len(cums_recomendados)} CUMs similares"
       }
       
   except Exception as e:
       logger.error(f"Error en recomendar_cums_similares: {str(e)}")
       return {
           "error": f"Error interno: {str(e)}",
           "cum_consultado": cum_input,
           "recomendaciones": []
       }

print("\n✅ FUNCIONES DE RECOMENDACIÓN DEFINIDAS")
print("🎯 Función principal: recomendar_cums_similares(cum_input)")


✅ FUNCIONES DE RECOMENDACIÓN DEFINIDAS
🎯 Función principal: recomendar_cums_similares(cum_input)


In [144]:
# 7. PRUEBAS DEL SISTEMA
# ============================================================================

print("\n🧪 PROBANDO SISTEMA CUM → CUMs...")

# Probar con 3 CUMs aleatorios
cums_prueba = df_final['CUM'].sample(n=3, random_state=123).tolist()

for i, cum_test in enumerate(cums_prueba, 1):
   print(f"\n🎬 PRUEBA {i}: CUM {cum_test}")
   print("-" * 50)
   
   resultado = recomendar_cums_similares(cum_test, df_final, 5)
   
   if "error" in resultado:
       print(f"❌ Error: {resultado['error']}")
       continue
   
   print(f"📋 CUM consultado: {resultado['cum_consultado']}")
   print(f"📋 Cluster: {resultado['cluster']}")
   print(f"📋 Mensaje: {resultado['mensaje']}")
   
   if resultado['recomendaciones']:
       print(f"\n💊 CUMs recomendados ({len(resultado['recomendaciones'])}):") 
       for j, cum_rec in enumerate(resultado['recomendaciones'], 1):
           print(f"   {j}. {cum_rec}")
   else:
       print("\n⚠️ Sin recomendaciones disponibles")



🧪 PROBANDO SISTEMA CUM → CUMs...

🎬 PRUEBA 1: CUM 19914757-1
--------------------------------------------------
📋 CUM consultado: 19914757-1
📋 Cluster: -1
📋 Mensaje: CUM en cluster de ruido - sin similares

⚠️ Sin recomendaciones disponibles

🎬 PRUEBA 2: CUM 19908174-2
--------------------------------------------------
📋 CUM consultado: 19908174-2
📋 Cluster: -1
📋 Mensaje: CUM en cluster de ruido - sin similares

⚠️ Sin recomendaciones disponibles

🎬 PRUEBA 3: CUM 19959949-14
--------------------------------------------------
📋 CUM consultado: 19959949-14
📋 Cluster: -1
📋 Mensaje: CUM en cluster de ruido - sin similares

⚠️ Sin recomendaciones disponibles


In [145]:
# 8. ESTADÍSTICAS FINALES
# ============================================================================

print("\n📊 ESTADÍSTICAS FINALES DEL SISTEMA")
print("="*45)

n_total = len(df_final)

print(f"📊 Total medicamentos: {n_total:,}")
print(f"🎯 Clusters generados: {resultados[mejor_algo]['n_clusters']}")
print(f"⚡ Algoritmo ganador: {mejor_algo.upper()}")

# Evaluar cobertura del sistema
print("\n🔍 Evaluando cobertura...")
sample_cums = df_final['CUM'].sample(n=min(500, len(df_final)), random_state=123)
con_recomendaciones = 0

for cum in sample_cums:
   resultado = recomendar_cums_similares(cum, df_final, 5)
   if resultado.get('recomendaciones', []):
       con_recomendaciones += 1

cobertura = (con_recomendaciones / len(sample_cums)) * 100
print(f"📈 Cobertura del sistema: {cobertura:.1f}%")
print(f"📋 Evaluados: {len(sample_cums):,} CUMs")

print("\n" + "="*60)
print("🎉 SISTEMA LISTO")
print("✅ Función: recomendar_cums_similares(cum_input)")
print("🎯 Input: CUM string → Output: Lista CUMs similares")
print("="*60)

logger.info("Sistema de recomendación por CUM completado exitosamente")


📊 ESTADÍSTICAS FINALES DEL SISTEMA
📊 Total medicamentos: 248,635
🎯 Clusters generados: 26
⚡ Algoritmo ganador: DBSCAN_ESTRICTO

🔍 Evaluando cobertura...


2025-06-19 19:40:10,359 - INFO - Sistema de recomendación por CUM completado exitosamente


📈 Cobertura del sistema: 2.2%
📋 Evaluados: 500 CUMs

🎉 SISTEMA LISTO
✅ Función: recomendar_cums_similares(cum_input)
🎯 Input: CUM string → Output: Lista CUMs similares


In [146]:
# 9. FUNCIÓN DE DEMOSTRACIÓN INTERACTIVA
# ============================================================================

def demo_sistema_cum(n_demos=3):
    """
    Demostración del sistema con CUMs reales
    """
    print(f"\n🎭 DEMOSTRACIÓN SISTEMA CUM → CUMs")
    print("="*40)
    
    # Seleccionar CUMs inválidos aleatorios
    demos = medicamentos_invalidos.sample(n=min(n_demos, len(medicamentos_invalidos)), 
                                          random_state=42)
    
    for i, (_, row) in enumerate(demos.iterrows(), 1):
        cum_demo = row['CUM']
        print(f"\n🔬 DEMO {i}: CUM {cum_demo}")
        print("-" * 30)
        
        resultado = recomendar_cums_similares(cum_demo, df_final, 5)
        
        print(f"📝 {resultado['mensaje']}")
        
        if resultado.get('recomendaciones'):
            print(f"💊 Recomendaciones:")
            for j, cum_rec in enumerate(resultado['recomendaciones'], 1):
                print(f"   {j}. {cum_rec}")
        
        if 'total_validos_cluster' in resultado:
            print(f"📊 Total válidos en cluster: {resultado['total_validos_cluster']}")

# Ejecutar demo
demo_sistema_cum(3)


🎭 DEMOSTRACIÓN SISTEMA CUM → CUMs

🔬 DEMO 1: CUM 20073338-2
------------------------------
📝 CUM en cluster de ruido - sin similares

🔬 DEMO 2: CUM 20016120-40
------------------------------
📝 CUM en cluster de ruido - sin similares

🔬 DEMO 3: CUM 20021002-9
------------------------------
📝 CUM en cluster de ruido - sin similares


In [147]:
# PROBAR CON UN CUM ESPECÍFICO
cum_test = "2203-1"  # Pon un CUM real del dataset
resultado = recomendar_cums_similares(cum_test, df_final, 5)
print(f"\n🔬 PRUEBA CON CUM: {cum_test}")
print(resultado)


🔬 PRUEBA CON CUM: 2203-1
{'cum_consultado': '2203-1', 'cluster': -1, 'mensaje': 'CUM en cluster de ruido - sin similares', 'recomendaciones': []}


In [126]:
df_encoded[df_encoded['CUM'] == '2203-1']

,CUM,ATC,VÍA ADMINISTRACIÓN,FORMA FARMACÉUTICA,PRINCIPIO ACTIVO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA,CANTIDAD CUM,UNIDAD,CANTIDAD,UNIDAD MEDIDA,EXPEDIENTE CUM
110370,2203-1,10.1902,18.3064,8.6413,943.0038,0,0,1,-0.050296,4.3729,-0.004135,6.2562,20062.0003


In [148]:
df_encoded[df_encoded['CUM'].isin(['20028480-1', '225029-2', '55684-1', '19926353-3', '20039895-1'])]

,CUM,ATC,VÍA ADMINISTRACIÓN,FORMA FARMACÉUTICA,PRINCIPIO ACTIVO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA,CANTIDAD CUM,UNIDAD,CANTIDAD,UNIDAD MEDIDA,EXPEDIENTE CUM
689,20028480-1,431.0109,4.2274,24.1578,1121.0032,0,0,1,-0.067695,3.9107,-0.004135,6.2562,20180.0003
891,225029-2,728.0053,18.3064,54.0356,644.0052,0,0,1,-0.085093,22.4076,-0.004135,6.2562,20069.0003
3458,55684-1,179.0312,18.3064,6.4681,755.0045,0,0,1,-0.071174,22.4076,-0.004135,19.1932,20089.0003
3576,19926353-3,1130.0022,4.2274,13.3975,1715.0022,0,0,1,-0.071174,3.9107,-0.004135,6.2562,20339.0003
5789,20039895-1,1009.0029,18.0435,5.7820,928.0038,0,0,1,-0.088573,22.4076,-0.004135,5.3227,20697.0002
143757,20028480-1,431.0109,4.2274,24.1578,1708.0022,0,0,1,-0.067695,3.9107,-0.004135,6.2562,20180.0003
184560,20039895-1,1009.0029,3.8369,5.7820,928.0038,0,0,1,-0.088573,22.4076,-0.004135,5.3227,20697.0002
209621,20028480-1,431.0109,4.2274,24.1578,1550.0024,0,0,1,-0.067695,3.9107,-0.004135,6.2562,20180.0003


## 7. Análisis de resultados y selección del mejor modelo

Analizamos los resultados de todos los algoritmos, creamos una tabla comparativa y seleccionamos el mejor modelo basado en las métricas de calidad. El mejor algoritmo será usado para el sistema de recomendación.